In [0]:
import json
from PIL import Image
import pandas as pd
import torch
from torchvision import transforms

Link to 

In [0]:
#Connecting google drive with colab
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [0]:


transform=transforms.Compose([
  transforms.CenterCrop((128,128)),
  transforms.ToTensor(),
 ])


#About the Dataset
In the Dataset conntains  the rendering parameters are given.
In each rendering folder, there is a metadata file for all images. Each row describes the parameters for an image. The first three numbers are pitch, roll, yaw, the fourth number is the distance between the center of the object and the camera. The fifth number is half of the camera FOV.

As the data contatins too many files and  its difficult to upload all, So i created a .npy folder.Link below in the text. \
Training_Label(Target) Data----https://drive.google.com/open?id=11oIPnNgjuyreib2hvoDPoQYYtOtBRpAB

Training_Images---https://drive.google.com/open?id=1gOaDUXZZAKImdNlMdjWtj3cjj9o4Q1Cw


In [0]:
 

import numpy as np

train_data=torch.tensor(np.load("/content/drive/My Drive/Data/images_200.npy"))#Loding traning data
train_label=(np.load("/content/drive/My Drive/Data/PoseEstimation_200.npy"))#Loading testing data which contatin s rotational data 
#####Padding 0's for shapeNet
pad=np.zeros((4,200,24,3))
train_label=torch.tensor(np.concatenate((train_label,pad),axis=3))#In ShapeNet dataset as images are CAD Designed So Translation(tx,ty,tz) is 0 and in the dataset the YAW(third angle) is also zero.

In [0]:
train_label1=train_label[:,:,:,:2]#Checking out the 2 angles.
train_label1

array([[[[327.83063  ,  28.339148 ],
         [ 81.787415 ,  26.986286 ],
         [ 92.53842  ,  26.447636 ],
         ...,
         [287.70966  ,  27.19946  ],
         [261.9943   ,  26.707312 ],
         [ 24.37572  ,  27.780745 ]],

        [[169.87946  ,  26.46758  ],
         [341.0166   ,  26.659391 ],
         [276.4787   ,  29.246212 ],
         ...,
         [340.47952  ,  28.376514 ],
         [ 57.655544 ,  26.799463 ],
         [185.12825  ,  29.92743  ]],

        [[297.69443  ,  28.096058 ],
         [309.6923   ,  26.328115 ],
         [169.68329  ,  26.88502  ],
         ...,
         [173.0978   ,  27.204275 ],
         [326.5601   ,  28.89589  ],
         [ 24.343206 ,  27.858917 ]],

        ...,

        [[ 81.33051  ,  29.966843 ],
         [ 25.968462 ,  26.442429 ],
         [296.63834  ,  25.248837 ],
         ...,
         [ 92.238815 ,  26.423494 ],
         [109.277855 ,  27.510363 ],
         [ 68.93004  ,  27.427126 ]],

        [[ 74.58996  ,  26.317463 

In [0]:
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import Linear,Conv2d,MaxPool2d,LeakyReLU,Tanh,Sigmoid

In [0]:

class GRUNet(nn.Module):
  def __init__(self):
        print('\nGruInitializing')
        super(GRUNet,self).__init__()
        
        self.is_x_tensor4 = False
        
        self.batch_size, self.img_w, self.img_h=1,128,128
        
        self.input_shape = (self.batch_size, 3, self.img_w, self.img_h)
        
        #number of filters for each convolution layer in the encoder
        self.n_convfilter = [96, 128, 256, 256, 256, 256]  
        
        #the dimension of the fully connected layer
        self.n_fc_filters = [1024]
        
       
        self.encoder=encoder(self.input_shape,self.n_convfilter,\
                            self.n_fc_filters)
        
       
  def forward(self,x):
    output= self.encoder(x)
    
    return output
      
  
 

#The Neural Architecture(Deep Res Net)
[Neural Architecture](https://user-images.githubusercontent.com/40520042/64719962-fe911b80-d4e6-11e9-88c0-6b941905de53.png)

In [0]:
#model(train_img[1].cuda())

class encoder(nn.Module):
  def __init__(self,input_shape,n_convfilter,\
               n_fc_filters):
        print("\ninitalizing \"encoder\"")
        super(encoder,self).__init__()
        #conv1
        self.conv1a = Conv2d(input_shape[1], n_convfilter[0], 7, padding=3,stride=1)#
        self.conv1b = Conv2d(n_convfilter[0], n_convfilter[0], 3, padding=1,stride=1)
        self.bn1 = nn.BatchNorm2d( n_convfilter[0])
        
        #conv2
        self.conv2a = Conv2d(n_convfilter[0], n_convfilter[1], 3, padding=1,stride=1)
        self.conv2b = Conv2d(n_convfilter[1], n_convfilter[1], 3, padding=1,stride=1)
        self.conv2c = Conv2d(n_convfilter[0], n_convfilter[1], 1)
        self.bn2 = nn.BatchNorm2d( n_convfilter[1])
        #conv3
        self.conv3a = Conv2d(n_convfilter[1], n_convfilter[2], 3, padding=1,stride=1)
        self.conv3b = Conv2d(n_convfilter[2], n_convfilter[2], 3, padding=1,stride=1)
        self.conv3c = Conv2d(n_convfilter[1], n_convfilter[2], 1)
        self.bn3 = nn.BatchNorm2d( n_convfilter[2])
        #conv4
        self.conv4a = Conv2d(n_convfilter[2], n_convfilter[3], 3, padding=1,stride=1)
        self.conv4b = Conv2d(n_convfilter[3], n_convfilter[3], 3, padding=1,stride=1)
        self.bn4 = nn.BatchNorm2d( n_convfilter[3])
        #conv5
        self.conv5a = Conv2d(n_convfilter[3], n_convfilter[4], 3, padding=1,stride=1)
        self.conv5b = Conv2d(n_convfilter[4], n_convfilter[4], 3, padding=1,stride=1)
        self.conv5c = Conv2d(n_convfilter[3], n_convfilter[4], 1)
        self.bn5 = nn.BatchNorm2d( n_convfilter[4])
        #conv6
        self.conv6a = Conv2d(n_convfilter[4], n_convfilter[5], 3, padding=1,stride=1)
        self.conv6b = Conv2d(n_convfilter[5], n_convfilter[5], 3, padding=1,stride=1)
        self.bn67 = nn.BatchNorm2d( n_convfilter[5])
        
        
        
        
        #pooling layer
        self.pool1 = MaxPool2d(kernel_size= 2,stride=2)#,return_indices=True)
        self.pool2 = MaxPool2d(kernel_size=1,stride=2)#,return_indices=True)
        #self.gpool=nn.AvgPool2d(kernel_size=2,stride=1)
        
        #nonlinearities of the network
        self.leaky_relu = LeakyReLU(negative_slope= 0.01)
        #self.sigmoid = Sigmoi
        self.tanh = Tanh()
        
        self.fc7 = Linear(1*1*256, 1024)
        self.fc8 = Linear(1024, 256)
        self.fc9 = Linear(256, 2)
            
        
            
  def forward(self, x):
        idx1,idx2,idx3=0,0,0
        #x is the input and the size of x is (batch_size, channels, heights, widths).
        
        conv1a = self.conv1a(x)
        rect1a = self.leaky_relu(conv1a)
        conv1b = self.conv1b(rect1a)
        
        rect1 = self.leaky_relu(conv1b)
        pool1= self.pool1(rect1)
        pool1=self.bn1(pool1)
        
        conv2a = self.conv2a(pool1)
        rect2a = self.leaky_relu(conv2a)
        conv2b = self.conv2b(rect2a)
        
        rect2 = self.leaky_relu(conv2b)
        conv2c = self.conv2c(pool1)
        res2 = conv2c + rect2
        pool2= self.pool2(res2)
        
        pool2=self.bn2(pool2)
        
        conv3a = self.conv3a(pool2)
        rect3a = self.leaky_relu(conv3a)
        conv3b = self.conv3b(rect3a)
        
        rect3 = self.leaky_relu(conv3b)
        conv3c = self.conv3c(pool2)
        res3 = conv3c + rect3
        
        pool3= self.pool2(res3)
        pool3=self.bn3(pool3)
        
        conv4a = self.conv4a(pool3)
        rect4a = self.leaky_relu(conv4a)
        conv4b = self.conv4b(rect4a)
        rect4 = self.leaky_relu(conv4b)
        pool4= self.pool2(rect4)
        
        pool4=self.bn4(pool4)
        
        conv5a = self.conv5a(pool4)
        rect5a = self.leaky_relu(conv5a)
        conv5b = self.conv5b(rect5a)
        rect5 = self.leaky_relu(conv5b)
        conv5c = self.conv5c(pool4)
        res5 = conv5c + rect5
        pool5= self.pool2(res5)
        
        pool5=self.bn5(pool5)
        
        conv6a = self.conv6a(pool5)
        rect6a = self.leaky_relu(conv6a)
        conv6b = self.conv6b(rect6a)
        rect6 = self.leaky_relu(conv6b)
        res6  = pool5 + rect6
        pool6 = self.pool2(res6)
        
        pool7=self.bn67(pool6)
        
       
        gpool=self.gpool(pool7)
      
        
        fc= gpool.view(gpool.size(0), -1)
       # print(fc.shape)
                
        fc7=self.fc7(fc)
        
        fc8=self.fc8(fc7)
        fc9=self.fc9(fc8)
        
       
        
        #print(rect7.shape)
        
        return fc9
    

In [0]:
#Object creation and testing
model=GRUNet().cuda()
out=model(train_data[1].cuda())



GruInitializing

initalizing "encoder"


In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion =  nn.L1Loss()# L1Loss is used 
# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)#Adam optimizer with learning rate of 0.001

Here we have taken batch size of 16 with 1200 in total.
As told translation angles are 0 and yaw is 0.
So lets estimate the other two.



In [0]:
print(train_data.shape)
print(train_label1.shape)
train_label1=torch.tensor(train_label1).view((1200,16,2))
train_data=train_data.view((1200,16,3,128,128))#

torch.Size([1200, 16, 3, 128, 128])
torch.Size([1200, 16, 2])


In [0]:
n_epochs = 20
#model=model.cuda()
valid_loss_min = np.Inf # track change in validation loss
model.train()
train_loss = 0.0
valid_loss=0.0
import datetime
for epoch in range(1, n_epochs+1):
  
    #scheduler.step() 
    print("EPOCH" ,epoch)
    #print("Learning Rate",scheduler.get_lr())
    train_loss = 0.0 
    valid_loss=0.0
    model.train()
    for i in range(len(train_data)):
          
                data=train_data[i].cuda()
                tar=train_label1[i].cuda()
                tar=tar.float()


                optimizer.zero_grad()

                output=model(data)

                loss = criterion(output, tar)

                loss.backward()
                optimizer.step()

                train_loss += loss.item()*data.size(0)

    print("ALL ABOUT LOSS(training)--------",(train_loss/(len(train_data))))             

"""    model.eval() 
    for j in range(len((list_tValid))):
          valid_img5=list_tValid[j]
          valid_target5=list_ValidL[j]         
    
          for i in range(len(valid_img5)):
                data=valid_img5[i].cuda()
                tar=valid_target5[i].cuda().long()
                loss = torch.mean(criterion(model(data), tar))
                valid_loss += loss.item()*data.size(0)
"""
      
      
#    print("ALL ABOUT LOSS(valid)--------",(valid_loss/(len(valid_target)*4)))
 #   print(" ")
      


EPOCH 1
ALL ABOUT LOSS(training)-------- 448.7939362271627
EPOCH 2
ALL ABOUT LOSS(training)-------- 362.1203946050008
EPOCH 3
ALL ABOUT LOSS(training)-------- 332.34549242019654
EPOCH 4
ALL ABOUT LOSS(training)-------- 312.6648017756144
EPOCH 5
ALL ABOUT LOSS(training)-------- 299.8508092021942
EPOCH 6
ALL ABOUT LOSS(training)-------- 287.09347827911375
EPOCH 7
ALL ABOUT LOSS(training)-------- 275.75534410476683
EPOCH 8
ALL ABOUT LOSS(training)-------- 265.95641799608865
EPOCH 9
ALL ABOUT LOSS(training)-------- 258.49188652356463
EPOCH 10
ALL ABOUT LOSS(training)-------- 249.10803530375162
EPOCH 11
ALL ABOUT LOSS(training)-------- 239.73272608121235
EPOCH 12
ALL ABOUT LOSS(training)-------- 232.19015185674033
EPOCH 13
ALL ABOUT LOSS(training)-------- 226.97227386156717
EPOCH 14
ALL ABOUT LOSS(training)-------- 221.0594281800588
EPOCH 15
ALL ABOUT LOSS(training)-------- 215.6503864256541
EPOCH 16
ALL ABOUT LOSS(training)-------- 212.82818539937338
EPOCH 17
ALL ABOUT LOSS(training)------

'    model.eval() \n    for j in range(len((list_tValid))):\n          valid_img5=list_tValid[j]\n          valid_target5=list_ValidL[j]         \n    \n          for i in range(len(valid_img5)):\n                data=valid_img5[i].cuda()\n                tar=valid_target5[i].cuda().long()\n                loss = torch.mean(criterion(model(data), tar))\n                valid_loss += loss.item()*data.size(0)\n'

In [0]:
model

GRUNet(
  (encoder): encoder(
    (conv1a): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (conv1b): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2a): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2c): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3c): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4a): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 